<a href="https://colab.research.google.com/github/victor7246/Notebooks/blob/master/Bert_fine_tune_on_bengalitexts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fine tune Transformer models on Bengali texts

In [1]:
# Install dependencies for Colab
#!pip uninstall -y tensorflow
!pip install transformers -U
!pip install tensorflow==1.15
#!pip uninstall -y tokenizers
!pip install tokenizers -U

Requirement already up-to-date: transformers in /usr/local/lib/python3.6/dist-packages (2.8.0)
  Using cached https://files.pythonhosted.org/packages/d1/3f/73c881ea4723e43c1e9acf317cf407fab3a278daab3a69c98dcac511c04f/tokenizers-0.5.2-cp36-cp36m-manylinux1_x86_64.whl
  Found existing installation: tokenizers 0.6.0
    Uninstalling tokenizers-0.6.0:
      Successfully uninstalled tokenizers-0.6.0
  Using cached https://files.pythonhosted.org/packages/73/de/ec55e2d5a8720557b25100dd7dd4a63108a44b6b303978ce2587666931cf/tokenizers-0.6.0-cp36-cp36m-manylinux1_x86_64.whl
ERROR: transformers 2.8.0 has requirement tokenizers==0.5.2, but you'll have tokenizers 0.6.0 which is incompatible.
  Found existing installation: tokenizers 0.5.2
    Uninstalling tokenizers-0.5.2:
      Successfully uninstalled tokenizers-0.5.2


In [64]:
ls ../

bin/                                       etc/    opt/    sys/
boot/                                      home/   proc/   tensorflow-1.15.2/
content/                                   lib/    root/   tmp/
datalab/                                   lib32/  run/    tools/
dev/                                       lib64/  sbin/   usr/
dlib-19.18.0-cp27-cp27mu-linux_x86_64.whl  media/  srv/    var/
dlib-19.18.0-cp36-cp36m-linux_x86_64.whl   mnt/    swift/


Load the dataset. In this notebook we are using texts collected from 4 popular Bengali novels. We extracted text using OCR from the scanned pdfs.

In [2]:
from google.colab import files


uploaded = files.upload()

Saving bengali_texts.txt to bengali_texts (1).txt


KeyboardInterrupt: ignored

In [6]:
uploaded.keys()

NameError: ignored

In [7]:
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer, SentencePieceBPETokenizer, BertWordPieceTokenizer
from transformers import *
import json

In [0]:
all_texts = open('./bengali_texts.txt','r').readlines()

In [9]:
print (all_texts[:3])

['আমতলি\n', '\n', 'রিদয় বলে ছেলেটা নামেই হৃদয়, দয়ামায়া একটুও ছিল না। পাখির\n']


We remove texts with < 3 words and do some whitespace cleaning

In [10]:
all_texts = [i.replace('\n','').strip() for i in all_texts]
normalized_texts = [i for i in all_texts if len(i.split()) >= 3]
print (len(normalized_texts))
print (normalized_texts[:3])

38689
['রিদয় বলে ছেলেটা নামেই হৃদয়, দয়ামায়া একটুও ছিল না। পাখির', 'ছুঁচোবাজি, কাকের ছানা ধরে তার নাকে তার দিয়ে নথ পরিয়ে দেওয়া, কুকুর-', 'ছানা বেরাল-ছানার ল্যাজে কাঁকড়া ধরিয়ে দেওয়া, ঘুমন্ত গুরুমহাশয়ের টিকিতে']


In [11]:
print (len(normalized_texts))

38689


In [0]:
with open('./normalized_bengali_texts.txt','w') as fp:
    for text in normalized_texts:
        fp.writelines(text+' \n')

In [0]:
#!wget https://s3.amazonaws.com/models.huggingface.co/bert/xlm-mlm-100-1280-vocab.json

In [0]:
!mkdir BengaliBERT

## Train a tokenizer

We will use Bert Wordpiece tokenization. We will train our own tokenizer.

In [11]:
'''
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files='./normalized_bengali_texts.txt', vocab_size=50000, min_frequency=4)
tokenizer.save(directory='./BengaliBERT',name='bengali-bpe')
'''

"\ntokenizer = ByteLevelBPETokenizer()\n\n# Customize training\ntokenizer.train(files='./normalized_bengali_texts.txt', vocab_size=50000, min_frequency=4)\ntokenizer.save(directory='./BengaliBERT',name='bengali-bpe')\n"

In [12]:
tokenizer = BertWordPieceTokenizer(
    vocab_file=None,
    add_special_tokens=True,
    unk_token='[UNK]',
    sep_token='[SEP]',
    cls_token='[CLS]',
    clean_text=True,
    handle_chinese_chars=True,
    strip_accents=True,
    lowercase=True,
    wordpieces_prefix='##'
)#SentencePieceBPETokenizer()

# Customize training
tokenizer.train(files='./normalized_bengali_texts.txt', vocab_size=50000, min_frequency=4, limit_alphabet=1000,
    special_tokens=['[PAD]', '[UNK]', '[CLS]', '[MASK]', '[SEP]'])

tokenizer.save(directory='./BengaliBERT',name='bengali')

['./BengaliBERT/bengali-vocab.txt']

In [0]:
!mv ./BengaliBERT/bengali-vocab.txt ./BengaliBERT/vocab.txt

Let us see how different tokenizers work

['দেখতে', '-', 'দেখতে', 'চালে', 'গিযে', 'গণেশের', 'মাথা', 'ঠেকল']

In [14]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
tokenizer.tokenize("দেখতে-দেখতে চালে গিয়ে গণেশের মাথা ঠেকল")

['দ',
 '##ে',
 '##খ',
 '##তে',
 '-',
 'দ',
 '##ে',
 '##খ',
 '##তে',
 'চ',
 '##াল',
 '##ে',
 'গ',
 '##িযে',
 'গ',
 '##ণ',
 '##ে',
 '##শের',
 'মা',
 '##থা',
 'ঠ',
 '##েক',
 '##ল']

In [15]:
tokenizer = XLMTokenizer.from_pretrained('xlm-mlm-100-1280')
tokenizer.tokenize("দেখতে-দেখতে চালে গিয়ে গণেশের মাথা ঠেকল")

['দেখ',
 'তে',
 '-',
 'দেখতে</w>',
 'চা',
 'লে</w>',
 'গিয</w>',
 '়</w>',
 'ে</w>',
 'গণ',
 'ে',
 'শের</w>',
 'মা',
 'থা</w>',
 'ঠ',
 'ে',
 'কল</w>']

In [16]:
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
tokenizer.tokenize("দেখতে-দেখতে চালে গিয়ে গণেশের মাথা ঠেকল")

['▁দেখতে',
 '-',
 'দে',
 'খ',
 'তে',
 '▁চাল',
 'ে',
 '▁গিয়ে',
 '▁গণ',
 'ে',
 'শ',
 'ের',
 '▁মাথা',
 '▁',
 'ঠ',
 'ে',
 'কল']

Below is our trained tokenizer

In [17]:
tokenizer = BertTokenizer.from_pretrained('./BengaliBERT')
tokenizer.tokenize("দেখতে-দেখতে চালে গিয়ে গণেশের মাথা ঠেকল")

['দেখতে', '-', 'দেখতে', 'চালে', 'গিযে', 'গণেশের', 'মাথা', 'ঠেকল']

## Train a language model from scratch

We will now train our language model using the [`run_language_modeling.py`](https://github.com/huggingface/transformers/blob/master/examples/run_language_modeling.py) script from `transformers` (newly renamed from `run_lm_finetuning.py` as it now supports training from scratch more seamlessly). Just remember to leave `--model_name_or_path` to `None` to train from scratch vs. from an existing model or checkpoint.

As the model is BERT-like, we’ll train it on a task of *Masked language modeling*, i.e. the predict how to fill arbitrary tokens that we randomly mask in the dataset. This is taken care of by the example script.


In [17]:
# Check that we have a GPU
!nvidia-smi

Wed Apr 15 12:21:59 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    25W / 250W |     10MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [13]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

Here, as we only have one text file, we don't even need to customize our `LineByLineDataset`. We'll just run the `run_language_modeling.py` script out-of-the-box.

In [20]:
# Get the example scripts.
!wget -c https://raw.githubusercontent.com/huggingface/transformers/master/examples/run_language_modeling.py

--2020-04-15 11:32:13--  https://raw.githubusercontent.com/huggingface/transformers/master/examples/run_language_modeling.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34411 (34K) [text/plain]
Saving to: ‘run_language_modeling.py’

run_language_modeli 100%[===================>]  33.60K  --.-KB/s    in 0.01s   

2020-04-15 11:32:13 (3.30 MB/s) - ‘run_language_modeling.py’ saved [34411/34411]



In [0]:
config = {
    "architectures": [
    "BertForMaskedLM"
    ],
    "attention_probs_dropout_prob": 0.1,
    "hidden_act": "gelu",
    "hidden_dropout_prob": 0.1,
    "hidden_size": 768,
    "initializer_range": 0.02,
    "intermediate_size": 3072,
    "layer_norm_eps": 1e-05,
    "max_position_embeddings": 514,
    "model_type": "bert",
    "num_attention_heads": 12,
    "num_hidden_layers": 6,
    "type_vocab_size": 1,
    "vocab_size": 52000
}
with open("./BengaliBERT/config.json", 'w') as fp:
    json.dump(config, fp)

tokenizer_config = {
    "max_len": 512
}
with open("./BengaliBERT/tokenizer_config.json", 'w') as fp:
    json.dump(tokenizer_config, fp)

In [42]:
!python run_language_modeling.py \
--train_data_file ./normalized_bengali_texts.txt \
--output_dir ./BengaliBERT-small \
--model_type bert \
--mlm \
--config_name ./BengaliBERT \
--tokenizer_name ./BengaliBERT \
--do_train \
--line_by_line \
--learning_rate 1e-4 \
--num_train_epochs 5 \
--save_total_limit 2 \
--save_steps 2000 \
--per_gpu_train_batch_size 16 \
--seed 42

04/15/2020 11:56:21 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
04/15/2020 11:56:21 - INFO - transformers.configuration_utils -   loading configuration file ./BengaliBERT/config.json
04/15/2020 11:56:21 - INFO - transformers.configuration_utils -   Model config BertConfig {
  "_num_labels": 2,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-05,
  "length_penalty": 1.0,
  "

In [0]:
!mkdir BengaliGPT2-small
!cp BengaliBERT/vocab.txt BengaliGPT2-small/vocab.txt

In [0]:
?GPT2Tokenizer.from_pretrained

In [0]:
config = {
    "architectures": [
    "GPT2LMHeadModel"
    ],
    "attention_probs_dropout_prob": 0.1,
    "hidden_act": "gelu",
    "hidden_dropout_prob": 0.1,
    "hidden_size": 768,
    "initializer_range": 0.02,
    "intermediate_size": 3072,
    "layer_norm_eps": 1e-05,
    "max_position_embeddings": 514,
    "model_type": "gpt2",
    "num_attention_heads": 12,
    "num_hidden_layers": 6,
    "type_vocab_size": 1,
    "vocab_size": 52000
}
with open("./BengaliGPT2-small/config.json", 'w') as fp:
    json.dump(config, fp)

tokenizer_config = {
    "max_len": 512,
    "eos_token": "</s",
    "unk_token": "<unk>",
    "sep_token": "<sep>",
    "pad_token": "<pad>",
    "cls_token": "<s>",
    "mask_token": "<mask>",
    "bos_token": "<bos>",
    "max_len_single_sentence": 512
}
with open("./BengaliGPT2-small/tokenizer_config.json", 'w') as fp:
    json.dump(tokenizer_config, fp)

In [96]:
!python run_language_modeling.py \
--train_data_file ./normalized_bengali_texts.txt \
--output_dir ./BengaliGPT2-small-v1 \
--model_type gpt2 \
--config_name ./BengaliGPT2-small \
--tokenizer_name ./BengaliGPT2-small \
--do_train \
--learning_rate 1e-4 \
--num_train_epochs 5 \
--save_total_limit 2 \
--save_steps 2000 \
--per_gpu_train_batch_size 16 \
--seed 42

04/15/2020 14:05:13 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
04/15/2020 14:05:13 - INFO - transformers.configuration_utils -   loading configuration file ./BengaliGPT2-small/config.json
04/15/2020 14:05:13 - ERROR - transformers.configuration_utils -   Can't set hidden_size with value 768 for GPT2Config {
  "_num_labels": 2,
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": 50256,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": 50256,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "length_penalty": 1.0,
  "max_length": 20,
  "min_length": 0,
  "model_type": "gpt2",
  "no_repe

Let us load the model and test

In [0]:
text = '[CLS] আমার মনটা [MASK] হয়ে গেল [SEP]'


In [0]:
import numpy as np

In [31]:
bert_model = BertForMaskedLM.from_pretrained('./BengaliBERT-small')
tokenizer = BertTokenizer.from_pretrained('./BengaliBERT-small',add_special_tokens=False)

indices = torch.LongTensor(np.asarray(tokenizer.encode(text,add_special_tokens=False))[np.newaxis,:])
tokens = tokenizer.tokenize(text)
print (tokens)
print (indices)
mask_indices = np.where(indices[0].numpy() == 3)[0]
print (mask_indices)

pred = bert_model(indices)
predicted_indices = np.argmax( np.asarray(pred[0][0].detach().numpy())[mask_indices,:] ,axis=1)
predicted_words_decoded = tokenizer.decode(predicted_indices)

print ("Predicted sentence -")
print (text.replace("[CLS]","").replace("[SEP]","").replace("[MASK]",predicted_words_decoded).strip())

n_top=20
top_10_indices = np.argsort(np.asarray(pred[0][0].detach().numpy())[mask_indices,:], axis=1)[0,-n_top:]
top_10_words = tokenizer.decode(top_10_indices).split()
top_10_words.reverse()

print ("Top 10 predicted words")
print (top_10_words)

['[CLS]', 'আমার', 'মনটা', '[MASK]', 'হযে', 'গেল', '[SEP]']
tensor([[   2,  224, 3005,    3,  243,  327,    4]])
[3]
Predicted sentence -
আমার মনটা । হয়ে গেল
Top 10 predicted words
['।', '-,', 'না', '“', 'করে', '|', 'আর', '_', 'তার', 'একটা', 'আমার', "থেকে!'আমি", 'যে', '##ে?র']


In [20]:
import gc
gc.collect()

280

In [21]:
!python run_language_modeling.py \
    --output_dir ./BengaliBERT-finetune-small \
    --model_type bert \
    --model_name_or_path bert-base-multilingual-uncased \
    --do_train \
    --mlm \
    --train_data_file ./normalized_bengali_texts.txt \
    --learning_rate 1e-4 \
    --num_train_epochs 3 \
    --save_total_limit 2 \
    --save_steps 2000 \
    --per_gpu_train_batch_size 4 \
    --seed 42

04/15/2020 12:30:19 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
04/15/2020 12:30:19 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-uncased-config.json from cache at /root/.cache/torch/transformers/33b56ce0f312e47e4d77a57791a4fc6233ae4a560dd2bdd186107058294e58ab.c7892120c5a9b21e515abc904e398dbabddf9510b122f659063cbf361fe16868
04/15/2020 12:30:19 - INFO - transformers.configuration_utils -   Model config BertConfig {
  "_num_labels": 2,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "directionality": "bidi",
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2labe

In [32]:
bert_model = BertForMaskedLM.from_pretrained('./BengaliBERT-finetune-small')
tokenizer = BertTokenizer.from_pretrained('./BengaliBERT-finetune-small',add_special_tokens=False)

indices = torch.LongTensor(np.asarray(tokenizer.encode(text,add_special_tokens=False))[np.newaxis,:])
tokens = tokenizer.tokenize(text)
print (tokens)
print (indices)
mask_indices = np.where(indices[0].numpy() == 103)[0]
print (mask_indices)

pred = bert_model(indices)
predicted_indices = np.argmax( np.asarray(pred[0][0].detach().numpy())[mask_indices,:] ,axis=1)
predicted_words_decoded = tokenizer.decode(predicted_indices)

print ("Predicted sentence -")
print (text.replace("[CLS]","").replace("[SEP]","").replace("[MASK]",predicted_words_decoded).strip())

n_top=20
top_10_indices = np.argsort(np.asarray(pred[0][0].detach().numpy())[mask_indices,:], axis=1)[0,-n_top:]
top_10_words = tokenizer.decode(top_10_indices).split()
top_10_words.reverse()

print ("Top 10 predicted words")
print (top_10_words)

['[CLS]', 'আ', '##মার', 'ম', '##নট', '##া', '[MASK]', 'হযে', 'গ', '##েল', '[SEP]']
tensor([[  101,   608, 60859,   642, 53712, 12198,   103, 24280,   620, 49802,
           102]])
[6]
Predicted sentence -
আমার মনটা শেষ হয়ে গেল
Top 10 predicted words
['শেষ', 'শর', 'না', 'কম', 'বড', 'মনে', 'ছোটওই', 'বেশি', 'নতন', 'কিছ', 'আর', 'বেশ', 'কথা', 'এই', 'তা', 'কারণ', 'তৈরি', 'ও']


In [34]:
gc.collect()

28

In [35]:
!python run_language_modeling.py \
    --output_dir ./BengaliXLM-finetune-small \
    --model_type xlmroberta \
    --model_name_or_path xlm-roberta-base \
    --do_train \
    --mlm \
    --train_data_file ./normalized_bengali_texts.txt \
    --learning_rate 1e-4 \
    --num_train_epochs 3 \
    --save_total_limit 2 \
    --save_steps 2000 \
    --per_gpu_train_batch_size 2 \
    --seed 42

04/15/2020 12:52:12 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
04/15/2020 12:52:12 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/xlm-roberta-base-config.json from cache at /root/.cache/torch/transformers/762ddd751172e9d3229e5da17a459eee6c0dfdc237c718944d0b1a85f06c7e1e.9ba214636e460976b286b4ce15e95d778f32439e9fdd8ddae7e3784f3a7e24a2
04/15/2020 12:52:12 - INFO - transformers.configuration_utils -   Model config XLMRobertaConfig {
  "_num_labels": 2,
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": 0,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": 2,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "

In [45]:
text = '<s> আমার মনটা <mask> হয়ে গেল </s>'

xlm_model = XLMRobertaForMaskedLM.from_pretrained('./BengaliXLM-finetune-small')
tokenizer = XLMRobertaTokenizer.from_pretrained('./BengaliXLM-finetune-small',add_special_tokens=False)

indices = torch.LongTensor(np.asarray(tokenizer.encode(text,add_special_tokens=False))[np.newaxis,:])
tokens = tokenizer.tokenize(text)
print (tokens)
print (indices)
mask_indices = np.where(indices[0].numpy() == 250001)[0]
print (mask_indices)

pred = xlm_model(indices)
predicted_indices = np.argmax( np.asarray(pred[0][0].detach().numpy())[mask_indices,:] ,axis=1)
predicted_words_decoded = tokenizer.decode(predicted_indices)

print ("Predicted sentence -")
print (text.replace("<mask>",predicted_words_decoded).strip())

n_top=20
top_10_indices = np.argsort(np.asarray(pred[0][0].detach().numpy())[mask_indices,:], axis=1)[0,-n_top:]
top_10_words = tokenizer.decode(top_10_indices).split()
top_10_words.reverse()

print ("Top 10 predicted words")
print (top_10_words)

['<s>', '▁আমার', '▁মন', 'টা', '<mask>', '▁হয়ে', '▁গেল', '</s>']
tensor([[     0,  29388, 102388,  11622, 250001,  25326, 130001,      2]])
[4]
Predicted sentence -
<s> আমার মনটা খারাপ হয়ে গেল </s>
Top 10 predicted words
['খারাপ', 'গরম', 'ভাল', 'বন্ধান্ত', 'জোররটকটাক', 'কঠিনলবল', 'বারও্ট', 'গোলালষ্টগ']


In [47]:
text = '<s> বিখ্যাত বৈজ্ঞানিক প্রোফেসর <mask> </s>'

xlm_model = XLMRobertaForMaskedLM.from_pretrained('./BengaliXLM-finetune-small')
tokenizer = XLMRobertaTokenizer.from_pretrained('./BengaliXLM-finetune-small',add_special_tokens=False)

indices = torch.LongTensor(np.asarray(tokenizer.encode(text,add_special_tokens=False))[np.newaxis,:])
tokens = tokenizer.tokenize(text)
print (tokens)
print (indices)
mask_indices = np.where(indices[0].numpy() == 250001)[0]
print (mask_indices)

pred = xlm_model(indices)
predicted_indices = np.argmax( np.asarray(pred[0][0].detach().numpy())[mask_indices,:] ,axis=1)
predicted_words_decoded = tokenizer.decode(predicted_indices)

print ("Predicted sentence -")
print (text.replace("<mask>",predicted_words_decoded).strip())

n_top=20
top_10_indices = np.argsort(np.asarray(pred[0][0].detach().numpy())[mask_indices,:], axis=1)[0,-n_top:]
top_10_words = tokenizer.decode(top_10_indices).split()
top_10_words.reverse()

print ("Top 10 predicted words")
print (top_10_words)

['<s>', '▁বি', 'খ্যাত', '▁বৈ', 'জ্ঞা', 'নিক', '▁প্র', 'ো', 'ফে', 'সর', '<mask>', '</s>']
tensor([[     0,   7277, 235591, 142351, 181597, 116605,  10066,   9445,  66272,
          65467, 250001,      2]])
[10]
Predicted sentence -
<s> বিখ্যাত বৈজ্ঞানিক প্রোফেসর অধ্যাপক </s>
Top 10 predicted words
['অধ্যাপক', 'জে', 'ডের', 'এড।র,েল', 'এমস্ু', 'খান.', 'টি', '-েনিয়ামান']


For Tensorflow we can run the below commands

In [0]:
#!git clone https://github.com/google-research/bert.git

In [51]:
'''
!python bert/create_pretraining_data.py \
    --input_file=./normalized_bengali_texts.txt \
    --output_file=./data_for_roberta.txt \
    --vocab_file=./BengaliBERT/bengali-vocab.txt \
    --do_lower_case=True \
    --max_seq_length=128 \
    --max_predictions_per_seq=20 \
    --masked_lm_prob=0.15 \
    --random_seed=42 \
    --dupe_factor=5
'''

'\n!python bert/create_pretraining_data.py     --input_file=./normalized_bengali_texts.txt     --output_file=./data_for_roberta.txt     --vocab_file=./BengaliBERT/bengali-vocab.txt     --do_lower_case=True     --max_seq_length=128     --max_predictions_per_seq=20     --masked_lm_prob=0.15     --random_seed=42     --dupe_factor=5\n'

In [52]:
'''
!python bert/run_pretraining.py \
    --input_file=./data_for_roberta.txt \
    --output_dir=./RobertaBengali/ \
    --do_train=True \
    --do_eval=True \
    --bert_config_file=./BengaliBERT/config.json \
    --train_batch_size=32 \
    --max_seq_length=128 \
    --max_predictions_per_seq=20 \
    --num_train_steps=200 \
    --num_warmup_steps=10 \
    --learning_rate=2e-5 \
    --use_tpu=False
'''

'\n!python bert/run_pretraining.py     --input_file=./data_for_roberta.txt     --output_dir=./RobertaBengali/     --do_train=True     --do_eval=True     --bert_config_file=./BengaliBERT/config.json     --train_batch_size=32     --max_seq_length=128     --max_predictions_per_seq=20     --num_train_steps=200     --num_warmup_steps=10     --learning_rate=2e-5     --use_tpu=False\n'

In [97]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./BengaliXLM-finetune-small",
    tokenizer="./BengaliXLM-finetune-small"
)

Model name './BengaliXLM-finetune-small' was not found in model name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased, bert-base-japanese, bert-base-japanese-whole-word-masking, bert-base-japanese-char, bert-base-japanese-char-whole-word-masking, bert-base-finnish-cased-v1, bert-base-finnish-uncased-v1, bert-base-dutch-cased, bart-large, bart-large-mnli, bart-large-cnn, bart-large-xsum, openai-gpt, transfo-xl-wt103, gpt2, gpt2-medium, gpt2-large, gpt2-xl, distilgpt2, ctrl, xlnet-base-cased, xlnet-large-cased, xlm-mlm-en-2048, xlm-mlm-ende-1024, xlm-mlm-enfr-1024, xlm-mlm-enro-1024

In [98]:
result = fill_mask("যেমন বসবার <mask>")
print (result)

[{'sequence': '<s> যেমন বসবার সময়</s>', 'score': 0.11186359822750092, 'token': 20895}, {'sequence': '<s> যেমন বসবার মতো</s>', 'score': 0.045684851706027985, 'token': 82829}, {'sequence': '<s> যেমন বসবার জন্য</s>', 'score': 0.03613507002592087, 'token': 8619}, {'sequence': '<s> যেমন বসবার উপায়</s>', 'score': 0.02854030579328537, 'token': 130095}, {'sequence': '<s> যেমন বসবার কথা</s>', 'score': 0.02673407830297947, 'token': 20559}]


In [99]:
result = fill_mask(" <mask> শঙ্কু ")
print (result)

[{'sequence': '<s>চন্দ্র শঙ্কু</s>', 'score': 0.14615142345428467, 'token': 233275}, {'sequence': '<s>র শঙ্কু</s>', 'score': 0.04195694997906685, 'token': 999}, {'sequence': '<s>ল শঙ্কু</s>', 'score': 0.03358595445752144, 'token': 2763}, {'sequence': '<s>ন শঙ্কু</s>', 'score': 0.027821704745292664, 'token': 2145}, {'sequence': '<s>রী শঙ্কু</s>', 'score': 0.021983271464705467, 'token': 40456}]


In [100]:
result = fill_mask(" <mask> ফেলুদা ")
print (result)

[{'sequence': '<s>, ফেলুদা</s>', 'score': 0.10524438321590424, 'token': 4}, {'sequence': '<s> আর ফেলুদা</s>', 'score': 0.06662540137767792, 'token': 16525}, {'sequence': '<s> বললেন ফেলুদা</s>', 'score': 0.04365352541208267, 'token': 191048}, {'sequence': '<s> ও ফেলুদা</s>', 'score': 0.029861927032470703, 'token': 2114}, {'sequence': '<s>। ফেলুদা</s>', 'score': 0.027037760242819786, 'token': 125}]
